In [11]:
import os
from uuid import uuid4

# The base setting of api
unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f" Corrector_agent_test_38"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__eb24c4743a814171b07dc0591921a299"  # Update to your API key
os.environ["TAVILY_API_KEY"] = "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"
# Used by the agent in this tutorial
os.environ["OPENAI_API_KEY"] = "sk-OqlsJwGTMdD1ujTG02Bb0fE08b7f4b30B07d8e83012bA8A8"
os.environ["OPENAI_API_BASE"] = "https://oneapi.xty.app/v1"

from langsmith import Client

client = Client()

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, BaseMessage, FunctionMessage, HumanMessage
import operator
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union

from langchain.agents import create_openai_functions_agent
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from tools.robot_func import python_repl

tools = [python_repl]
# corrector_prompt
corrector_prompt = hub.pull("blackj/corrector_prompt")
# print(corrector_prompt)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


# corrector_prompt
# tools = []
# print(corrector_prompt.messages)


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    corrector_state: bool


corrector_template = \
    """You job is to make the user_requirenment clear.

Just Output the Result,dont output decription.Dont make judge but help. 

You should work as the procedure including in the ###

### 1. Interpret the Command of user: Analyze the user's input to determine if they want the robot to "Get" something or "Go" somewhere. Be aware that abbreviations might be used. 

Example: Interpret "GO" as "go to", and "MV" as "move to". ###

###

2. Correct and Clarify: Look for any spelling errors in the user's command that might indicate a specific location or item, it might something or somewhere in the house. Clarify the command without adding extra information. Note: Focus solely on the user's input for this step. Action: Correct any spelling mistakes and clarify the meaning of places or items mentioned.

Example: User Input: "Robt, GO to the kithcn and gt me a sppon." Corrected Command: "Robot,go to the kitchen and get me a spoon." ###
"""


def correct_user_input(state:AgentState):
    last_message = state["messages"]
    if state["corrector_state"]:
        return [SystemMessage(
            content=f"The user has confirm the command,I will send it to robot!!,the corrector command is {state['message'][-2]}",
            kwarg={"corrector_state": True})] + last_message
    else:
        return [SystemMessage(content=corrector_template)] + last_message

def parse(ai_message: AIMessage) -> AgentState:
    """Parse the AI message."""
    return {"messages":ai_message.content}

# llm_with_tool = llm.bind_tools(tools)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# 
# corrector_chain = correct_user_input | llm | StrOutputParser()
# corrector_test = (
#     {"user_requirenment": lambda x: x["input"]}
#         | corrector_prompt 
#         | llm 
#         | StrOutputParser())
# 
# corrector_test.invoke({"input":"robot,get tea"})



NameError: name 'AIMessage' is not defined